# INITALIZATION

In [0]:
import pyspark.sql.functions as F
from pyspark.sql.window import Window


# Business Logic

In [0]:
dim_products_sql = """
WITH base AS (
  SELECT
    CAST(product_key AS STRING) AS product_key,
    product_name,
    category,
    subcategory,
    brand,
    ROW_NUMBER() OVER (PARTITION BY CAST(product_key AS STRING) ORDER BY product_name) AS rn
  FROM silver.crm_products
)
SELECT
  xxhash64(product_key) AS product_sk,
  product_key,
  product_name,
  category,
  subcategory,
  brand
FROM base
WHERE rn = 1
"""

# Write the Gold Table

In [0]:
dim_products_sql = """
WITH base AS (
  SELECT
    CAST(product_key AS STRING) AS product_key,
    product_name,
    category_id,
    ROW_NUMBER() OVER (PARTITION BY CAST(product_key AS STRING) ORDER BY product_name) AS rn
  FROM workspace.silver.crm_products
)
SELECT
  xxhash64(product_key) AS product_sk,
  product_key,
  product_name,
  category_id
FROM base
WHERE rn = 1
"""

# Write the Gold Layer

In [0]:
dim_products_df = spark.sql(dim_products_sql)

(dim_products_df.write
 .mode("overwrite")
 .format("delta")
 .saveAsTable("workspace.gold.dim_products"))

In [0]:
%sql
SELECT * FROM workspace.gold.dim_products